## IMPORT

In [1]:
update = 1

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from settings import *
import time
import pandas as pd
import gspread
from tkinter.messagebox import showinfo
import configparser

window = tk.Tk()
# def var_callback(var, index, mode):
#     # print(var, index, mode)
#     window.update()
#     upload_csv_line.update()

# csv_track = csv_file.trace_add('write', var_callback)
# json_track = json_file.trace_add('write', var_callback)
csv_df = pd.DataFrame()

In [2]:
cfg = configparser.ConfigParser()
cfg.read('cfg.ini')

K = {}

for key in cfg['VAR']:
    K[key] = StringVar()
    K[key].set(cfg['VAR'][key])
    print(key)

print(K['list_var'].get())

last_directory
csv_file
json_file
google_sheet_name
accept_button
verify_button
compare_button
export_button
close_button
load_output
list_var
selected_columns
fillament
sort_by
google_len
['Entry ID', 'Что для Вас есть авторская песня', 'Страна, город', 'Возраст', 'Место учёбы или работы', 'Адрес электронной почты', 'Согласие на обработку персональных данных', 'Custom HTML', 'Custom HTML.1', 'Название номера', 'Название коллектива и исполнители', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие)', 'Автор слов в Вашем выступлении', 'Custom HTML.2', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие).1', 'Автор музыки в Вашем выступлении', 'Как Вы узнали о Гордеевском фестивале?', 'IP Address', 'Date']


In [3]:
print(K['last_directory'])
print(K['last_directory'])

PY_VAR12
PY_VAR12


In [4]:
# for key in cfg['VAR']:
    

# ADJUSTING WINDOW

In [5]:
window.resizable(0, 0)
window.title('CSV2GSheets')
window.geometry('800x400')

window.columnconfigure(0, weight=3)
window.columnconfigure(1, weight=2)
window.rowconfigure(0)

def CloseWindow():
    window.quit()
    window.destroy()
    
def UpdateWindow():
    time.sleep(1)
    window.update_idletasks()

frame_left = ttk.Frame(window, width=15, height=4)
frame_left.grid(column=0, row=0, sticky=tk.W, padx=0, pady=5)


frame_left.columnconfigure(0, weight=5)
frame_left.columnconfigure(1, weight=1)



# json picker
frame_left.rowconfigure(0, weight=1)
frame_left.rowconfigure(1, weight=1)

csv_df = tk.Variable()

def uploadCSV(event=None):
    filename = filedialog.askopenfilename(
        initialdir=K['last_directory'].get() or 'sensitive/',
        title="Open your CSV file",
        filetypes=(("CSV files", "*.csv"),))
    K['csv_file'].set(filename)
    upload_csv_line.configure(text=K['csv_file'].get())
    # Open CSV to compare their lenght
    csv_df = pd.read_csv(K['csv_file'].get())
    csv_len = len(csv_df['Entry ID'])
    K['list_var'].set(csv_df.columns.tolist())
    print(csv_df.columns.tolist())
    
upload_csv_label = ttk.Label(frame_left, text="CSV file", width=150, justify='center')
upload_csv_label.grid(column=1, row=0, sticky=tk.NSEW, padx=0, pady=5, columnspan=1)
upload_csv_line = ttk.Label(frame_left, text=K['csv_file'].get(), wraplength=300, justify='left', borderwidth=1, relief='sunken', width=50)
upload_csv_line.grid(column=1, row=1, padx=0, pady=5, sticky=tk.NSEW)
upload_csv_button = tk.Button(frame_left, text='Browse', command=uploadCSV, width=20, justify='right')
upload_csv_button.grid(column=2, row=1, padx=0, pady=5, sticky=tk.NSEW)



frame_left.rowconfigure(2, weight=1)
frame_left.rowconfigure(3, weight=1)

def uploadJSON():
    filename = filedialog.askopenfilename(
        initialdir=K['last_directory'].get() or 'sensitive/',
        title="Open your CSV file",
        filetypes=(("JSON files", "*.json"),))
    K['json_file'].set(filename)
    upload_json_line.configure(text=K['json_file'].get())
    UpdateWindow()
    sheets_name_entry.delete(0, tk.END)
    sheets_name_entry.insert(tk.END, google_sheet_name.get())
    print(K['google_sheet_name'].get())


upload_google_label = ttk.Label(frame_left, text="JSON file", width=8, justify='left')
upload_google_label.grid(column=1, row=2, sticky=tk.NSEW, padx=0, pady=5, columnspan=1)
upload_json_line = ttk.Label(frame_left, text=K['json_file'].get(), wraplength=300, justify='left', borderwidth=1, relief='sunken', width=50)
upload_json_line.grid(column=1, row=3, padx=0, pady=5, sticky=tk.NSEW)
upload_google_button = tk.Button(frame_left, text='Browse', command=uploadJSON, width=20, justify='right')
upload_google_button.grid(column=2, row=3, padx=0, pady=5, sticky=tk.NSEW)

sheets_name_label = ttk.Label(frame_left, text="Google Sheet name", width=8, justify='left')
sheets_name_label.grid(column=1, row=4, sticky=tk.NSEW, padx=0, pady=5, columnspan=1)

# gc = gspread.service_account(K['json_file'].get())
# workbooks = [file["name"] for file in gc.list_all_spreadsheets()]
# variable_list = workbooks


# for i in variable_list:
#     listbox.insert(END, i)
    
# sheets_name_entry = tk.Listbox(window, selectmode=SINGLE, height = len(variable_list))
# sheets_name_entry.grid(column=1, row=5, padx=0, pady=5, sticky=tk.NSEW)

# sheets_name_entry = ttk.Entry(frame_left, textvariable=K['google_sheet_name'], justify='left', width=50, background='green')
# sheets_name_entry.grid(column=1, row=5, padx=0, pady=5, sticky=tk.NSEW)
sheets_name_entry = ttk.Entry(frame_left, textvariable=K['google_sheet_name'], justify='left', width=50, background='green')
sheets_name_entry.grid(column=1, row=5, padx=0, pady=5, sticky=tk.NSEW)
sheets_name_update = tk.Button(frame_left, text='Browse', command=uploadJSON, width=20, justify='right')
sheets_name_update.grid(column=2, row=5, padx=0, pady=5, sticky=tk.NSEW)

frame_left.rowconfigure(4, weight=1)
frame_left.rowconfigure(5, weight=1)



def load_tables():
    upload_csv_line.configure(text=K['csv_file'].get())
    # Open CSV to compare their lenght
    csv_df = (pd.read_csv(K['csv_file'].get()))
    csv_len = len(csv_df['Entry ID'])
    K['list_var'].set(csv_df.columns.tolist())
    
    google_df = pd.DataFrame()
    # Pull Google Sheet (should be allowed)
    # to compare their lenght
    gc = gspread.service_account(filename=K['json_file'].get())
    google_sheet = gc.open('VIII Гордеевский фестиваль')
    # Notice that it points to sheet1. You can change it.
    google_df = pd.DataFrame(google_sheet.sheet1.get_all_values())
    google_len = len(google_sheet.sheet1.col_values(1))-1

    # Set the first row from GS as column names
    google_df.columns = google_df.iloc[0]
    google_df = google_df[1:]

    # Fill NaN and sort values to match
    # you can customize this value
    fill_with = K['fillament'].get()
    sort_by = K['sort_by'].get()

    csv_df = pd.read_csv(K['csv_file'].get())
    csv_len = len(csv_df['Entry ID'])
    K['list_var'].set(csv_df.columns.tolist())
    

    
    google_df  = google_df.fillna(fill_with)
    csv_df = csv_df.fillna(fill_with)
    csv_df = csv_df.sort_values(by=[sort_by], ascending=True)

    
    if csv_df.columns.all() == google_df.columns.all():
        columns_len = len(csv_df.columns.all())

    
        # Calculate difference between their lenghtes
        print('# of strings in GSS: ', google_len)
        print('# of strings in CSV: ', csv_len)
        difference = csv_len - google_len
        print('They differ by', abs(difference))
        if difference > 0:
            update_button_label.configure(text='Ready to update Google Tables [+'+str(difference)+' entries]')
            update_button['state'] = 'normal'
        elif difference < 0:
            update_button_label.configure(text='Your CSV has less entries than Google Sheets. Wrong CSV?')
        else: update_button_label.configure(text='Tables are in sync. No actions needed.')
        
    else: print('Column labels don\'t match. Wrong tables?')
    
def update_table():
    gc = gspread.service_account(filename=K['json_file'].get())
    google_sheet = gc.open('VIII Гордеевский фестиваль')
    # Notice that it points to sheet1. You can change it.
    google_df = pd.DataFrame(google_sheet.sheet1.get_all_values())
    google_len = len(google_sheet.sheet1.col_values(1))-1

    # Set the first row from GS as column names
    google_df.columns = google_df.iloc[0]
    google_df = google_df[1:]

    # Fill NaN and sort values to match
    # you can customize this value
    fill_with = K['fillament'].get()
    sort_by = K['sort_by'].get()
    
    csv_df = pd.read_csv(K['csv_file'].get())
    csv_len = len(csv_df['Entry ID'])
    K['list_var'].set(csv_df.columns.tolist())
    
    google_df  = google_df.fillna(fill_with)
    csv_df = csv_df.fillna(fill_with)
    csv_df = csv_df.sort_values(by=[sort_by], ascending=True)
    
    
    current_row = google_len + 1
    difference = csv_len - google_len + 1
    for i in range(difference):
        taken_row = csv_df.iloc[current_row-google_len-2]
        
        # constructing an Excel-style pointer
        diff_range = str('A' + str(current_row) + ':S' + str(current_row))
        
        # it double checks if'd write into an empty row
        if google_sheet.sheet1.cell(current_row,1).value == None:
            taken_row = taken_row.values.tolist()
            print('values', taken_row, 'range', diff_range)
            for pos in range(len(taken_row)):
                taken_row[pos] = str(taken_row[pos])
            google_sheet.sheet1.update(
                values=[taken_row],
                range_name = str(diff_range))
        current_row+=1
        
    K['load_output'].set('Tables are in sync.')

update_button_label = ttk.Label(frame_left, text="Please compare tables", wraplength=350, justify='right', borderwidth=1, relief='ridge', width=75)
update_button_label.grid(column=1, row=6, padx=0, pady=5, columnspan=2, sticky=tk.EW)
load_tables_button = tk.Button(frame_left, text='Compare tables', command=load_tables)
load_tables_button.grid(column=1, row=7, sticky=tk.NSEW, padx=0, pady=5)



# close button
frame_left.rowconfigure(6, weight=1)




frame_right = ttk.Frame(window, width=15, height=4)
frame_right.grid(column=3, row=0)

frame_right.columnconfigure(0, weight=1)
frame_right.columnconfigure(1, weight=1)

frame_right.rowconfigure(0, weight=1)
frame_right.rowconfigure(1, weight=1)
frame_right.rowconfigure(2, weight=1)

frame_right.rowconfigure(3, weight=1)
frame_right.rowconfigure(4, weight=1)
# def select_all():
#     lb.select_set(0, tk.END)

# lb = tk.Listbox(frame_right, selectmode='MULTIPLE')
# for i in range(10): lb.insert(tk.END, i)
# lb.pack()
# tk.Button(frame_right, text='select all', command=select_all).pack()

# frame_right.columnconfigure(0, weight=1)
# frame_right.columnconfigure(1, weight=1)

# upload_google_label = ttk.Label(frame_right, text="JSON\nfile", width=5, justify='left')
# upload_google_label.grid(column=0, row=1, sticky=tk.E, padx=0, pady=5)
# upload_json_line = ttk.Label(frame_right, text=json_file.get(), wraplength=350, justify='left', borderwidth=1, relief='sunken', width=75)
# upload_json_line.grid(column=1, row=1, padx=0, pady=5)
print(list_var.get())

('Entry ID', 'Что для Вас есть авторская песня', 'Страна, город', 'Возраст', 'Место учёбы или работы', 'Адрес электронной почты', 'Согласие на обработку персональных данных', 'Custom HTML', 'Custom HTML.1', 'Название номера', 'Название коллектива и исполнители', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие)', 'Автор слов в Вашем выступлении', 'Custom HTML.2', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие).1', 'Автор музыки в Вашем выступлении', 'Как Вы узнали о Гордеевском фестивале?', 'IP Address', 'Date')


# ESTABLISHING FUNCTIONS

In [6]:



    

    
def items_selected(event):
    # get all selected indices
    selected_indices = listbox.curselection()
    print('s.i', selected_indices)
    # get selected items
    K['selected_columns'].set(",".join([listbox.get(i) for i in selected_indices]))
    print(selected_columns.get())

    listbox.bind('<<ListboxSelect>>', items_selected)
    print('tolist', csv_df.columns.tolist())
    



def transform_column_ids():
    print('selected columns', selected_columns.get(), 'selected_columns')

# EMBEDDING WIDGETS

In [7]:




# Right side

listbox_label = ttk.Label(frame_right, text="Select columns", width=8, justify=tk.CENTER)
listbox_label.grid(column=1, row=0, sticky=tk.EW, padx=0, pady=5)
listbox = tk.Listbox(frame_right, listvariable=list_var.get(), height=10, selectmode=tk.EXTENDED, width = 50, bg='white')
listbox.grid(column=1, row=1, sticky=tk.EW, padx=0, pady=5)
list_accept = tk.Button(frame_right, text='Accept', command=transform_column_ids)
list_accept.grid(column=1, row=2, sticky=tk.EW, padx=0, pady=5)
listbox_label = ttk.Label(frame_right, text="", width=8, justify=tk.CENTER)
listbox_label.grid(column=1, row=3, sticky=tk.EW, padx=0, pady=5)
update_button = tk.Button(frame_right, text='Update\ntable', command=update_table)
# update_button = tk.Button(frame_right, text='Update\ntable', command=update_table, state='disabled')
update_button.grid(column=1, row=4, sticky=tk.NSEW, padx=0, pady=5)
close_button = tk.Button(frame_right, text='Close', command=CloseWindow)
close_button.grid(column=1, row=5, sticky=tk.NS, padx=0, pady=5, columnspan=2)



# END

In [8]:
# csv_df = pd.read_csv('sensitive/фестиваль-авторской-песни-им-алексан-weforms-entries-1701780626.csv')
# display(csv_df.head())
# print(str(last_directory.get()+json_file.get()))
# json_df = pd.read_json('sensitive/gordeevtable-8880501906d7.json', typ='series')
# display(json_df)

In [9]:
window.mainloop()

# csv_file.trace_remove('write', csv_track)
# json_file.trace_remove('write', json_track)
print('END')

['Entry ID', 'Что для Вас есть авторская песня', 'Страна, город', 'Возраст', 'Место учёбы или работы', 'Адрес электронной почты', 'Согласие на обработку персональных данных', 'Custom HTML', 'Custom HTML.1', 'Название номера', 'Название коллектива и исполнители', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие)', 'Автор слов в Вашем выступлении', 'Custom HTML.2', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие).1', 'Автор музыки в Вашем выступлении', 'Как Вы узнали о Гордеевском фестивале?', 'IP Address', 'Date']
VIII Гордеевский фестиваль
Column labels don't match. Wrong tables?
END


In [10]:
# from tkinter import *

# def select_all():
#     lb.select_set(0, END)

# root = Tk()
# lb = Listbox(root, selectmode=MULTIPLE)
# for i in range(10): lb.insert(END, i)
# lb.pack()
# Button(root, text='select all', command=select_all).pack()
# root.mainloop()

In [11]:
if update == 1:
    import configparser
    cfg = configparser.ConfigParser()

    cfg['VAR'] = {
        'last_directory' : ('sensitive'),
    'csv_file' : (''),
    'json_file' : (''),
    'google_sheet_name' : ('VIII Гордеевский фестиваль'),

    'accept_button' : (''),
    'verify_button' : (''),
    'compare_button' : (''),
    'export_button' : (''),
    'close_button' : (''),
    'load_output' : ('none'),
    'list_var' : (['Entry ID', 'Что для Вас есть авторская песня', 'Страна, город', 'Возраст', 'Место учёбы или работы', 'Адрес электронной почты', 'Согласие на обработку персональных данных', 'Custom HTML', 'Custom HTML.1', 'Название номера', 'Название коллектива и исполнители', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие)', 'Автор слов в Вашем выступлении', 'Custom HTML.2', 'Ссылка на запись Вашего выступления (YouTube, Google Drive, OneDrive и другие).1', 'Автор музыки в Вашем выступлении', 'Как Вы узнали о Гордеевском фестивале?', 'IP Address', 'Date']),
    'selected_columns' : ([]),
    'fillament' : ('-'),
    'sort_by' : ('Entry ID'),
    'google_len' : (3)
    }
    print('sex')
    
    print(cfg['VAR']['google_len'])

    with open('cfg.ini', 'w') as configfile:
        cfg.write(configfile)

sex
3


# ======================

In [12]:
# import tkinter as tk
# from tkinter import ttk
# from tkinter.messagebox import showinfo

# # create the root window
# root = tk.Tk()
# root.title('Listbox')


# # create a list box
# langs = ('Java', 'C#', 'C', 'C++', 'Python',
#          'Go', 'JavaScript', 'PHP', 'Swift')

# var = tk.Variable(value=langs)

# listbox = tk.Listbox(
#     root,
#     listvariable=var,
#     height=6,
#     selectmode=tk.MULTIPLE
# )

# listbox.pack(expand=True, fill=tk.BOTH)


# def items_selected(event):
#     # get all selected indices
#     selected_indices = listbox.curselection()
#     # get selected items
#     selected_langs = ",".join([listbox.get(i) for i in selected_indices])
#     msg = f'You selected: {selected_langs}'
#     showinfo(title='Information', message=msg)


# listbox.bind('<<ListboxSelect>>', items_selected)

# root.mainloop()

In [13]:
# import tkinter as tk
# from tkinter import ttk
# from tkinter import filedialog
# from settings import *
# import time

# def var_callback(var, index, mode):
#     # print(var, index, mode)
#     window.update()
#     window.update_idletasks()
#     upload_csv_line.update()
#     upload_google_line.update()


# csv_track = csv_file.trace_add('write', var_callback)
# json_track = json_file.trace_add('write', var_callback)

# window = tk.Tk()

# window.resizable(0, 0)
# window.title('CSV2GSheets')
# window.geometry('800x600') 

# # csv picker
# window.columnconfigure(0, weight=1)
# window.columnconfigure(1, weight=3)
# window.columnconfigure(2, weight=1)

# #json picker
# window.rowconfigure(0, weight=1)
# window.rowconfigure(1, weight=1)
# window.rowconfigure(2, weight=5)

# # operations

# #exit
# window.rowconfigure(3, weight=1)

# def uploadCSV(event=None):
#     filename = filedialog.askopenfilename(
#         initialdir=last_directory.get() or 'sensitive/',
#         title="Open your CSV file",
#         filetypes=(("CSV files", "*.csv"),))
#     csv_file.set(filename)
#     upload_csv_label.config(csv_file.get())
#     print(csv_file.get())
#     updateWindow()


# def uploadJSON(event=None):
#     filename = filedialog.askopenfilename(
#         initialdir=last_directory.get() or 'sensitive/',
#         title="Open your CSV file",
#         filetypes=(("JSON files", "*.json"),))
#     json_file.set(filename)
#     upload_google_label.config(json_file.get())
#     updateWindow()

 
# def CloseWindow():
#     window.quit()
#     window.destroy()
    
# def updateWindow():
#     # time.sleep(1)
#     window.update_idletasks()

# upload_csv_label = ttk.Label(window, text="Your CSV file:")
# upload_csv_label.grid(column=0, row=0, sticky=tk.W, padx=0, pady=5)
# upload_csv_line = ttk.Label(window, text=csv_file.get())
# upload_csv_line.grid(column=1, row=0, sticky=tk.W, padx=0, pady=5)
# upload_csv_button = tk.Button(window, text='Browse', command=uploadCSV)
# upload_csv_button.grid(column=2, row=0, sticky=tk.E, padx=0, pady=5)


# upload_google_label = ttk.Label(window, text="Your JSON file:")
# upload_google_label.grid(column=0, row=1, sticky=tk.W, padx=0, pady=5)
# upload_google_line = ttk.Label(window, text=json_file.get() or last_directory)
# upload_google_line.grid(column=1, row=1, sticky=tk.W, padx=0, pady=5)
# upload_google_button = tk.Button(window, text='Browse', command=uploadJSON)
# upload_google_button.grid(column=2, row=1, sticky=tk.E, padx=0, pady=5)

# close_button = tk.Button(window, text='Close', command=CloseWindow)
# close_button.grid(column=3, row=3, sticky=tk.SE, padx=0, pady=5)

# window.mainloop()

# csv_file.trace_remove('write', csv_track)
# json_file.trace_remove('write', json_track)
# print('END')

In [14]:
# from tkinter import *

# win = Tk()

# x = IntVar()
# x.set(1)

# def add():
#     x.set(x.get() + 1)

# label = Label(win, textvariable=x)
# label.pack()

# button = Button(win, text="Increment", command=add)
# button.pack()

# win.mainloop()

In [15]:
# from tkinter import *

# def multiplier(*args):
#     try:
#         value = float(ment.get())
#         res = value *2
#         result.set(res)
#         mEntry.delete(0, END) #deletes the current value
#         mEntry.insert(0, res) #inserts new value assigned by 2nd parameter

#     except ValueError:
#         pass

# mGui = Tk()
# mGui.geometry("300x300+300+300")

# ment = StringVar()
# result = StringVar()

# mbutton = Button (mGui, text = "Calculate", command = multiplier)
# mbutton.pack()

# mEntry = Entry(mGui, textvariable = ment, text="bebe")
# mEntry.pack()

# mresult = Label(mGui, textvariable = result)
# mresult.pack()

# mGui.mainloop()

In [16]:
# tf = filedialog.askopenfilename(
#         initialdir="C:/Users/MainFrame/Desktop/", 
#         title="Open Text file", 
#         filetypes=(("Text Files", "*.txt"),)
#         )

In [17]:
# from tkinter import Tk
# from tkinter.filedialog import askopenfilename

# Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
# filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
# print(filename)
# quit()

In [18]:
# import tkinter as tk
# from tkinter import filedialog as fd

# def get_file():
#     file = fd.askopenfile()
#     if file:
#         print(file.name)
#         root.destroy(root)

# root = tk.Tk

# root.button_1 = tk.Button(text="Click me!",
#     width=25,
#     height=50,
#     bg="blue",
#     fg="yellow",
#     command=get_file())

# root.button_2 = tk.Button(root, text="Quit", command=lambda root=root:quit()).pack()

# root.mainloop()

In [19]:
# import tkinter as tk

# def kill(x):
#     x.quit()
#     x.destroy()
    
# def quit(self):
#     self.destroy()
#     exit()

# root = tk.Tk()
# button = tk.Button(
#     text="Click me!",
#     width=25,
#     height=50,
#     bg="blue",
#     fg="yellow",
#     command=quit(root)
# )
# greeting = tk.Label(text="Hello, Tkinter")
# greeting.pack()
# button.pack()

# root.mainloop()

In [20]:
# from tkinter import Tk, Label
# root = Tk()
# w = Label(root, text="Hello, world!")
# w.pack()
# root.mainloop()

In [21]:
# import pandas as pd
# import numpy as np
# import sys 
# from tkinter import * 
# root = Tk()
# root.geometry('580x250')
# dates = pd.date_range('20210101', periods=8)
# dframe = pd.DataFrame(np.random.randn(8,4),index=dates,columns=list('ABCD'))
# txt = Text(root)
# txt.pack()

# class PrintToTXT(object): 
#     def write(self, s):
#         txt.insert(END, s)

# sys.stdout = PrintToTXT()

# print ('Pandas date range of 8 values in 1 timestamp column adjacent to a numpy random float array of 8 rows and 4 columns, displayed in a Tkinter table') 
# print (dframe)
# root.mainloop()

In [ ]:
# import tkinter as tk

# window = tk.Tk()

# frame_a = tk.Frame()
# frame_b = tk.Frame()

# label_a = tk.Label(master=frame_a, text="I'm in Frame A")
# label_a.pack()

# label_b = tk.Label(master=frame_b, text="I'm in Frame B")
# label_b.pack()

# frame_a.pack()
# frame_b.pack()

# window.mainloop()


In [ ]:
# from tkinter import *
# window=tk.Tk()

# # add widgets here
# v1 = IntVar()
# v2 = IntVar()
# C1 = Checkbutton(window, text = "Cricket", variable = v1)
# C2 = Checkbutton(window, text = "Tennis", variable = v2)
# C1.place(x=100, y=100)
# C2.place(x=180, y=100)

# window.title('Hello Python')
# window.geometry("300x200")
# window.mainloop()

# if v1 == True:
#     print('yep')

In [ ]:
# from tkinter import *
# class MyWindow:
#     def __init__(self, win):
#         self.lbl1=Label(win, text='First number')
#         self.lbl2=Label(win, text='Second number')
#         self.lbl3=Label(win, text='Result')
#         self.t1=Entry(bd=3)
#         self.t2=Entry()
#         self.t3=Entry()
#         self.btn1 = Button(win, text='Add')
#         self.btn2=Button(win, text='Subtract')
#         self.lbl1.place(x=100, y=50)
#         self.t1.place(x=200, y=50)
#         self.lbl2.place(x=100, y=100)
#         self.t2.place(x=200, y=100)
#         self.b1=Button(win, text='Add', command=self.add)
#         self.b2=Button(win, text='Subtract')
#         self.b2.bind('<Button-1>', self.sub)
#         self.b1.place(x=100, y=150)
#         self.b2.place(x=200, y=150)
#         self.lbl3.place(x=100, y=200)
#         self.t3.place(x=200, y=200)
#     def add(self):
#         self.t3.delete(0, 'end')
#         num1=int(self.t1.get())
#         num2=int(self.t2.get())
#         result=num1+num2
#         self.t3.insert(END, str(result))
#     def sub(self, event):
#         self.t3.delete(0, 'end')
#         num1=int(self.t1.get())
#         num2=int(self.t2.get())
#         result=num1-num2
#         self.t3.insert(END, str(result))

# window=Tk()
# mywin=MyWindow(window)
# window.title('Hello Python')
# window.geometry("400x300")
# window.mainloop()

In [ ]:
# # Check if columns' list and labels matches -
# # this script would stop otherwise
# if csv_df.columns.all() == google_df.columns.all():
    

#     # Check if our CSV has new rows over GSheets
#     if difference > 0:
#         print("They differ by ", difference, ' strings!')

#     # If yes, we push new lines to GSheets in a cycle
#         current_row = google_len + 1

#         for i in range(difference+1):
#             taken_row = csv_df.iloc[current_row-2].astype('str')
            
#             # constructing an Excel-style pointer
#             diff_range = str('A' + str(current_row) + ':S' + str(current_row))
            
#             # it double checks if'd write into an empty row
#             if google_sheet.sheet1.cell(current_row,1).value == None:
#                 google_sheet.sheet1.update(values =
#                                         [taken_row.values.tolist()],
#                                         range_name = diff_range)
#             current_row+=1
#         print('Tables are in sync.')
    
#     # If it's the oposite or they are equal,
#     # nothing would happen   
#     elif difference < 0:
#         print('Your CSV has less entries than Google Sheets. Wrong SCV?')
#     else: print('Tables are in sync. No actions needed.')

# else: print ('Column labels don\'t match. Wrong tables?')